

Latent Dirichlet Allocation (LDA) es un modelo generativo que permite que conjuntos de observaciones puedan ser explicados por grupos no observados que explican por qué algunas partes de los datos son similares. Por ejemplo, si las observaciones son palabras en documentos, presupone que cada documento es una mezcla de un pequeño número de categorías (también denominados como tópicos) y la aparición de cada palabra en un documento se debe a una de las categorías a las que el documento pertenece. LDA es un ejemplo de modelo de categorías




Cargamos los datos de los tickets de dcip

In [2]:
import sys
import IPython
import numpy as np
import pandas as pd
import sklearn as sk


import warnings
warnings.filterwarnings('ignore')
import pandas as pd

df = pd.read_csv("datasets/tickets.csv")


df.head(4)

,subject,content
0,jenkins dejado funcionar,partir horas dejado funcionar sospecha causas ...
1,usuario para artefactos artifactory,necesito artifact estamos generando departamen...
2,permisos carpetas dcip,podriais permisos carpetas eleven paths dcip 1...
3,repo artifactory,repositorio artifactory para guardar imagenes ...


## Hacemos el preprocesado ##
 - Quitamos las stop words
 - Vectorizamos el texto 

In [3]:
#CountVectorizer implements both tokenization and occurrence counting
#  in a single class:


from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['content'].values)
print(X)

  (0, 1990)	1
  (0, 794)	3
  (0, 396)	3
  (0, 1440)	3
  (0, 989)	3
  (0, 323)	3
  (0, 876)	3
  (0, 4403)	3
  (0, 2096)	5
  (0, 1393)	5
  (0, 2309)	3
  (0, 3420)	3
  (1, 4658)	1
  (1, 4635)	1
  (1, 4089)	1
  (1, 515)	3
  (1, 1101)	3
  (1, 4767)	6
  (1, 725)	6
  (1, 3802)	6
  (1, 696)	3
  (1, 3999)	3
  (1, 2136)	3
  (1, 4820)	4
  (1, 4736)	6
  :	:
  (1069, 2397)	2
  (1069, 2292)	2
  (1069, 4283)	2
  (1069, 2192)	3
  (1069, 4687)	2
  (1069, 3652)	2
  (1069, 4030)	2
  (1069, 4800)	2
  (1069, 4207)	2
  (1069, 1165)	2
  (1069, 3155)	3
  (1069, 1176)	2
  (1069, 1524)	2
  (1069, 2139)	2
  (1069, 1168)	3
  (1069, 2509)	3
  (1069, 1779)	4
  (1069, 3384)	3
  (1069, 4889)	5
  (1069, 4376)	2
  (1069, 4585)	2
  (1069, 4523)	2
  (1069, 2312)	2
  (1069, 1262)	3
  (1069, 2846)	2


Sacamos los tópicos de cada ticket

In [26]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=5,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [1]:
lda.components_.shape
lda.head(3)

NameError: name 'lda' is not defined

In [23]:
n_top_words = 6
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
ituser help contacta marcossuarez contraseña ayuda
Topic 2:
argos splunkdeploy instance amd64 saas splunk
Topic 3:
imagen android nuevo crear esclavo repositorio
Topic 4:
java hudson received plugins origin messaging
Topic 5:
plugin documento aubay antes esclavo estamos


In [18]:

mytopic = X_topics[:, 4].argsort()[::-1]
print ("Caso", mytopic)
for iter_idx, movie_idx in enumerate(mytopic[:8]):
    print('\nTopic #%d:' % (iter_idx + 1))
    print(df['subject'][movie_idx][:300], '...')
    print(df['content'][movie_idx][:300], '...')
    


Caso [950 948 805 ... 445 429 868]

Topic #1:
hook pdihub jenkins ...
mientras concluida validada migracion todos proyectos dcip queremos mantener compilacion ambos entornos dcip nuestros jenkins plataforma pdihub permite añadir webhook tipo repositorio actualmente para compilacion nuestro jenkins esta activado webhook github plugin funciona correctamente dado posible ...

Topic #2:
jenkins jira plugin funciona ...
jira podemos sacar releasenotes proyecto soprpm mediante consultas jira estando usuario autenticado dado queremos utilizar autenticacion basica queriamos utilizar plugin jenkins para generar releasenotes embargo obtienen ningun resultado muestra ningun error pantalla nuestro proyecto ejemplo dcip te ...

Topic #3:
dcip solicitud instalacion plugin upstream changes ...
vendria bien instalar plugin dcip para poder monitorizar cambios proyecto padre dado lugar build pipeline wiki jenkinsci jenkins upstream changes plugin salu2 aubay marcos pindado sebastian aubay spain juan her

In [7]:
# Use tf-idf features for NMF.

from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

n_samples = 5000
n_features = 1000
n_components = 10
n_top_words = 20

data_samples = df.content[:n_samples]


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()



print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(df['content'])
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()



# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)



# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
done in 0.217s.
Extracting tf features for LDA...
done in 0.211s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=5000 and n_features=1000...
done in 0.139s.

Topics in NMF model (Frobenius norm):
Topic #0: borrada maquinas sido queue action creator amaris listas epgmad9 resuelto borrar resolucion comenzara pondra contigo pronto volvera respondiendo posible borrado
Topic #1: docker server team access usage possible support deployment configured work client export issues remember password security environment information version host
Topic #2: dcip para jenkins acceso proyecto usuario maquina esta jobs resuelto github permisos posible sido artifactory pdihub pero esclavo mismo muchas
Topic #3: ituser contacta help attached inform host command tefdigital returned basecamp citrix backups ones lync puppet necesitas jira best ayuda connection
Topic #4: argos splunkdeploy instance saas access information splunk openstack finish en